# LangChain 核心模块：Data Conneciton - Document Transformers

一旦加载了文档，通常会希望对其进行转换以更好地适应您的应用程序。

最简单的例子是，您可能希望将长文档拆分为较小的块，以适应模型的上下文窗口。LangChain具有许多内置的文档转换器，可以轻松地拆分、合并、过滤和其他操作文档。


![](https://python.langchain.com/assets/images/data_connection-c42d68c3d092b85f50d08d4cc171fc25.jpg)

### Document 类

这段代码定义了一个名为`Document`的类，允许用户与文档的内容进行交互，可以查看文档的段落、摘要，以及使用查找功能来查询文档中的特定字符串。

```python
# 基于BaseModel定义的文档类。
class Document(BaseModel):
    """接口，用于与文档进行交互。"""

    # 文档的主要内容。
    page_content: str
    # 用于查找的字符串。
    lookup_str: str = ""
    # 查找的索引，初次默认为0。
    lookup_index = 0
    # 用于存储任何与文档相关的元数据。
    metadata: dict = Field(default_factory=dict)

    @property
    def paragraphs(self) -> List[str]:
        """页面的段落列表。"""
        # 使用"\n\n"将内容分割为多个段落。
        return self.page_content.split("\n\n")

    @property
    def summary(self) -> str:
        """页面的摘要（即第一段）。"""
        # 返回第一个段落作为摘要。
        return self.paragraphs[0]

    # 这个方法模仿命令行中的查找功能。
    def lookup(self, string: str) -> str:
        """在页面中查找一个词，模仿cmd-F功能。"""
        # 如果输入的字符串与当前的查找字符串不同，则重置查找字符串和索引。
        if string.lower() != self.lookup_str:
            self.lookup_str = string.lower()
            self.lookup_index = 0
        else:
            # 如果输入的字符串与当前的查找字符串相同，则查找索引加1。
            self.lookup_index += 1
        # 找出所有包含查找字符串的段落。
        lookups = [p for p in self.paragraphs if self.lookup_str in p.lower()]
        # 根据查找结果返回相应的信息。
        if len(lookups) == 0:
            return "No Results"
        elif self.lookup_index >= len(lookups):
            return "No More Results"
        else:
            result_prefix = f"(Result {self.lookup_index + 1}/{len(lookups)})"
            return f"{result_prefix} {lookups[self.lookup_index]}"
```


## Text Splitters 文本分割器

当你想处理长篇文本时，有必要将文本分成块。听起来很简单，但这里存在着潜在的复杂性。理想情况下，你希望将语义相关的文本片段放在一起。"语义相关"的含义可能取决于文本类型。这个笔记本展示了几种实现方式。

从高层次上看，文本分割器的工作原理如下：

1. 将文本分成小而有意义的块（通常是句子）。
2. 开始将这些小块组合成较大的块，直到达到某个大小（通过某个函数进行测量）。
3. 一旦达到该大小，使该块成为自己独立的一部分，并开始创建一个具有一定重叠（以保持上下文关系）的新文本块。

这意味着您可以沿两个不同轴向定制您的文本分割器：

1. 如何拆分文字
2. 如何测量块大小

### 使用 `RecursiveCharacterTextSplitter` 文本分割器

该文本分割器接受一个字符列表作为参数，根据第一个字符进行切块，但如果任何切块太大，则会继续移动到下一个字符，并以此类推。默认情况下，它尝试进行切割的字符包括 `["\n\n", "\n", " ", ""]`

除了控制可以进行切割的字符外，您还可以控制其他一些内容：

- length_function：用于计算切块长度的方法。默认只计算字符数，但通常在这里传递一个令牌计数器。
- chunk_size：您的切块的最大大小（由长度函数测量）。
- chunk_overlap：切块之间的最大重叠部分。保持一定程度的重叠可以使得各个切块之间保持连贯性（例如滑动窗口）。
- add_start_index：是否在元数据中包含每个切块在原始文档中的起始位置。

#### 分割文本

In [1]:
# 加载待分割长文本
with open('../../tests/state_of_the_union.txt') as f:
    state_of_the_union = f.read()

In [2]:
print(state_of_the_union)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their bodies. Every

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)

In [5]:
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])
print(texts[1])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and' metadata={'start_index': 0}
page_content='of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.' metadata={'start_index': 82}


In [6]:
type(texts)

list

In [7]:
type(texts[0])

langchain.schema.document.Document

In [8]:
state_of_the_union[:1000]

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [9]:
metadatas = [{"document": 1}, {"document": 2}]
documents = text_splitter.create_documents([state_of_the_union, state_of_the_union], metadatas=metadatas)
print(documents[0])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and' metadata={'document': 1, 'start_index': 0}


#### 分割代码

In [10]:
from langchain.text_splitter import Language

In [11]:
# 支持编程语言的完整列表
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'js',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol']

In [12]:
html_text = """
<!DOCTYPE html>
<html>
    <head>
        <title>🦜️🔗 LangChain</title>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            h1 {
                color: darkblue;
            }
        </style>
    </head>
    <body>
        <div>
            <h1>🦜️🔗 LangChain</h1>
            <p>⚡ Building applications with LLMs through composability ⚡</p>
        </div>
        <div>
            As an open source project in a rapidly developing field, we are extremely open to contributions.
        </div>
    </body>
</html>
"""

In [13]:
html_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.HTML, chunk_size=60, chunk_overlap=0
)
html_docs = html_splitter.create_documents([html_text])

In [15]:
print(len(html_docs))

13


In [16]:
html_docs

[Document(page_content='<!DOCTYPE html>\n<html>', metadata={}),
 Document(page_content='<head>\n        <title>🦜️🔗 LangChain</title>', metadata={}),
 Document(page_content='<style>\n            body {\n                font-family: Aria', metadata={}),
 Document(page_content='l, sans-serif;\n            }\n            h1 {', metadata={}),
 Document(page_content='color: darkblue;\n            }\n        </style>\n    </head', metadata={}),
 Document(page_content='>', metadata={}),
 Document(page_content='<body>', metadata={}),
 Document(page_content='<div>\n            <h1>🦜️🔗 LangChain</h1>', metadata={}),
 Document(page_content='<p>⚡ Building applications with LLMs through composability ⚡', metadata={}),
 Document(page_content='</p>\n        </div>', metadata={}),
 Document(page_content='<div>\n            As an open source project in a rapidly dev', metadata={}),
 Document(page_content='eloping field, we are extremely open to contributions.', metadata={}),
 Document(page_content='</di